## Analyze e2e latencies for the single_client_e2e experiment

In [4]:
import numpy as np
import glob

def get_append_metrics(path):
    file_pattern = path + "append_metrics*.csv"

    total_throughput = 0
    latency_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                gsn, latency, throughput = int(parts[0]), float(parts[1]), float(parts[2])
                latency_values.append(latency)
            
            total_throughput += throughput

    latency_array = np.array(latency_values)

    mean_latency = np.mean(latency_array)
    p50_latency = np.percentile(latency_array, 50)
    p99_latency = np.percentile(latency_array, 99)

    print("results for computation time " + path.split("_")[-1].split("/")[0] + " us")
    print("statistic/metric, latency (us)")
    print(f"mean, {mean_latency:.2f}")
    print(f"p50, {p50_latency:.2f}")
    print(f"p99, {p99_latency:.2f}")
    print(f"total throughput, {total_throughput:.2f}")

    return int(path.split("_")[-1].split("/")[0]), mean_latency, total_throughput 

def get_e2e_metrics(path):
    file_pattern = path + "e2e_metrics*.csv"

    delivery_latency_values = []
    e2e_latency_values = []
    queuing_delay_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                delivery, e2e, queuing_delay = float(parts[1]), float(parts[2]), float(parts[3])
                delivery_latency_values.append(delivery)
                e2e_latency_values.append(e2e)
                queuing_delay_values.append(queuing_delay)

    
    delivery_latency_array = np.array(delivery_latency_values)
    e2e_latency_array = np.array(e2e_latency_values)
    queuing_delay_array = np.array([x for x in queuing_delay_values if x > 0])

    print("statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)")
    print(f"mean, {np.mean(delivery_latency_array):.2f}, {np.mean(e2e_latency_array):.2f}, {np.mean(queuing_delay_array):.2f}")
    print(f"std, {np.std(delivery_latency_array):.2f}, {np.std(e2e_latency_array):.2f}, {np.std(queuing_delay_array):.2f}")
    print(f"p50, {np.percentile(delivery_latency_array, 50):.2f}, {np.percentile(e2e_latency_array, 50):.2f}, {np.percentile(queuing_delay_array, 50):.2f}")
    print(f"p99, {np.percentile(delivery_latency_array, 99):.2f}, {np.percentile(e2e_latency_array, 99):.2f}, {np.percentile(queuing_delay_array, 99):.2f}")

    return int(path.split("_")[-1].split("/")[0]), np.mean(delivery_latency_array), np.mean(e2e_latency_array), np.mean(queuing_delay_array)

In [5]:
path = "../results/"

for dir in glob.glob(path + "*/"):
    get_append_metrics(dir)
    get_e2e_metrics(dir)


In [ ]:
import re
import pandas as pd

# Input data
data = """
results for computation time 100 us
statistic/metric, latency (us)
mean, 3413.93
p50, 3157.00
p99, 8432.00
total throughput, 19516.55
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3715.53, 3888.02, 71.50
std, 1257.74, 1257.86, 35.13
p50, 3443.00, 3616.00, 91.00
p99, 9182.00, 9356.00, 110.00
results for computation time 200 us
statistic/metric, latency (us)
mean, 3430.22
p50, 3157.00
p99, 8519.23
total throughput, 19451.48
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3732.03, 4083.27, 149.21
std, 1438.34, 1462.64, 120.47
p50, 3454.00, 3806.00, 183.00
p99, 9130.00, 9528.00, 207.00
results for computation time 800 us
statistic/metric, latency (us)
mean, 3465.14
p50, 3210.00
p99, 8585.00
total throughput, 19473.98
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3808.53, 5084.88, 474.60
std, 1304.37, 1352.79, 270.25
p50, 3547.00, 4855.00, 519.00
p99, 9243.00, 10616.00, 804.00
results for computation time 1500 us
statistic/metric, latency (us)
mean, 3347.53
p50, 3071.00
p99, 8479.65
total throughput, 19494.47
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3663.30, 5939.39, 769.03
std, 1296.19, 1394.54, 448.66
p50, 3357.00, 5725.00, 763.00
p99, 9178.00, 11633.00, 1500.00
results for computation time 2500 us
statistic/metric, latency (us)
mean, 3435.22
p50, 3108.00
p99, 8949.00
total throughput, 19432.59
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3718.95, 7495.61, 1265.89
std, 1349.38, 1556.19, 730.11
p50, 3378.00, 7323.00, 1267.00
p99, 9621.00, 13646.00, 2496.00
results for computation time 1000 us
statistic/metric, latency (us)
mean, 3412.40
p50, 3171.00
p99, 8322.92
total throughput, 19502.57
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3730.30, 5289.09, 552.50
std, 1283.03, 1362.40, 332.08
p50, 3477.00, 5021.00, 559.00
p99, 9045.00, 10760.00, 1001.00
results for computation time 3000 us
statistic/metric, latency (us)
mean, 3609.59
p50, 3472.00
p99, 8543.00
total throughput, 19504.00
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3900.91, 8421.15, 1507.33
std, 1214.04, 1526.28, 880.59
p50, 3762.00, 8297.00, 1497.00
p99, 9167.00, 14054.00, 2994.00
results for computation time 2000 us
statistic/metric, latency (us)
mean, 3356.87
p50, 3075.00
p99, 8226.00
total throughput, 19535.13
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3665.72, 6685.76, 1011.05
std, 1224.57, 1381.11, 587.51
p50, 3373.00, 6523.00, 1020.00
p99, 9001.88, 12240.00, 1997.00
results for computation time 2200 us
statistic/metric, latency (us)
mean, 3247.09
p50, 2971.00
p99, 8783.86
total throughput, 19471.79
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3562.10, 6886.25, 1114.31
std, 1306.12, 1476.66, 643.43
p50, 3245.00, 6706.00, 1114.00
p99, 9375.00, 12960.00, 2197.00
results for computation time 3800 us
statistic/metric, latency (us)
mean, 3580.25
p50, 3412.00
p99, 8330.00
total throughput, 19513.60
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3886.28, 9618.77, 1917.79
std, 1273.71, 1703.48, 1107.22
p50, 3714.00, 9513.00, 1919.00
p99, 9041.73, 15101.00, 3789.00
results for computation time 2800 us
statistic/metric, latency (us)
mean, 3649.67
p50, 3566.00
p99, 8316.00
total throughput, 19527.79
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3924.91, 8150.01, 1414.24
std, 1197.35, 1467.99, 815.83
p50, 3835.00, 8024.00, 1414.00
p99, 9037.00, 13509.00, 2793.00
results for computation time 1200 us
statistic/metric, latency (us)
mean, 3751.53
p50, 3717.00
p99, 8587.00
total throughput, 19487.41
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 4054.26, 5886.57, 627.04
std, 1286.73, 1356.94, 366.79
p50, 3995.00, 5712.00, 618.00
p99, 9265.00, 11259.00, 1201.00
results for computation time 3300 us
statistic/metric, latency (us)
mean, 3477.94
p50, 3233.00
p99, 8355.24
total throughput, 19479.81
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3795.72, 8779.79, 1668.92
std, 1431.71, 1759.55, 971.96
p50, 3552.00, 8647.00, 1668.00
p99, 9087.27, 14379.00, 3292.00
results for computation time 5000 us
statistic/metric, latency (us)
mean, 3528.66
p50, 3412.00
p99, 8739.00
total throughput, 19467.53
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3852.41, 11400.05, 2523.06
std, 1310.05, 2106.13, 1527.92
p50, 3732.00, 11297.00, 2514.00
p99, 9313.00, 17333.00, 4984.00
results for computation time 500 us
statistic/metric, latency (us)
mean, 3195.12
p50, 2963.00
p99, 8764.00
total throughput, 19459.88
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3499.87, 4326.88, 327.78
std, 1376.39, 1394.78, 181.01
p50, 3222.00, 4101.00, 409.00
p99, 9373.00, 10255.17, 507.00
results for computation time 4000 us
statistic/metric, latency (us)
mean, 3315.87
p50, 3032.00
p99, 8592.00
total throughput, 19504.46
statistic/metric, delivery latency (us), e2e latency (us), queuing delay (us)
mean, 3601.12, 9636.16, 2017.90
std, 1267.28, 1755.86, 1172.32
p50, 3293.00, 9512.00, 2024.00
p99, 9240.71, 15696.00, 3991.00

"""

# Split data into lines
lines = data.splitlines()

# Parsing logic
results = []
current_time = None
append_latency = None

for i, line in enumerate(lines):
    line = line.strip()
    # Detect computation time
    if line.startswith("results for computation time"):
        match = re.search(r"computation time (\d+) us", line)
        if match:
            current_time = int(match.group(1))
    # Extract append latency (first "latency (us)" mean value)
    elif "statistic/metric, latency (us)" in line:
        append_line = lines[i + 1]  # Look at the next line
        append_match = re.match(r"mean,\s*([\d.]+)", append_line)
        if append_match:
            append_latency = float(append_match.group(1))
    # Extract other latencies (mean values)
    elif line.startswith("mean,") and current_time is not None:
        parts = line.split(",")
        if len(parts) == 4:  # Ensure correct format
            delivery_latency, e2e_latency, queueing_delay = map(float, parts[1:])
            results.append({
                "computation_time": current_time,
                "append_latency": append_latency,
                "delivery_latency": delivery_latency,
                "e2e_latency": e2e_latency,
                "queueing_delay": queueing_delay,
            })

# Create a DataFrame
df = pd.DataFrame(results)

df.to_csv("output.csv", index=False)

# Print the CSV content
print(df.to_csv(index=False))